In [62]:
p_storage_dir = "../../storage"

In [63]:
import pandas as pd
import re
from pathlib import Path

DA_STORAGE_DIR = Path(f"{p_storage_dir}/dados_abertos")

# Cursos

In [64]:
cursos = pd.read_csv(DA_STORAGE_DIR / "transformed/cursos.csv", delimiter=";")

cursos.head()

,campus,nome,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
0,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Análises ...,967,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3683,3186.0,120.0,160.0,108.0,NaN,3574.0
1,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Meio Ambi...,968,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3684,2970.0,120.0,160.0,54.0,NaN,3304.0
2,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Vigilânci...,908,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3592,3240.0,120.0,160.0,54.0,NaN,3574.0
3,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Enfermage...,909,Integrado - EJA,Presencial,Noturno,Semestral,60,Ensino Médio,3593,2547.0,120.0,NaN,NaN,NaN,2667.0
4,Câmpus Águas Lindas,Licenciatura em Ciências Biológicas,1174,Licenciatura,Presencial,Noturno,Anual,30,Superior,4316,2960.0,200.0,NaN,108.0,NaN,3268.0


# Substituição do nome do câmpus pela sigla

Para garantir que o curso seja ligado corretamente com o câmpus em que é/foi ofertado, o nome do câmpus foi substituído pela sigla.

In [65]:
for campus in cursos["campus"].unique():
    print(f"• {campus}")

• Câmpus Águas Lindas
• Câmpus Anápolis
• Câmpus Aparecida de Goiânia
• Câmpus Cidade de Goiás
• Câmpus Formosa
• Câmpus Goiânia
• Câmpus Goiânia Oeste
• Câmpus Inhumas
• Câmpus Itumbiara
• Câmpus Jataí
• Câmpus Luziânia
• Câmpus Senador Canedo
• Câmpus Uruaçu
• Câmpus Valparaíso
• Câmpus Virtual de Ead


In [66]:
campus_replacement_map = {
    "Câmpus Águas Lindas": "LIN",
    "Câmpus Anápolis": "ANA",
    "Câmpus Aparecida de Goiânia": "APA",
    "Câmpus Cidade de Goiás": "GOI",
    "Câmpus Formosa": "FOR",
    "Câmpus Goiânia": "GYN",
    "Câmpus Goiânia Oeste": "OES",
    "Câmpus Inhumas": "INH",
    "Câmpus Itumbiara": "ITU",
    "Câmpus Jataí": "JAT",
    "Câmpus Luziânia": "LUZ",
    "Câmpus Senador Canedo": "SEN",
    "Câmpus Uruaçu": "URU",
    "Câmpus Valparaíso": "VAL"
}

In [67]:
cursos["campus"].replace(campus_replacement_map, inplace=True)

In [68]:
for campus in cursos["campus"].unique():
    print(f"• {campus}")

• LIN
• ANA
• APA
• GOI
• FOR
• GYN
• OES
• INH
• ITU
• JAT
• LUZ
• SEN
• URU
• VAL
• Câmpus Virtual de Ead


# Excluir os cursos do câmpus EAD

Atualmente existem 5 cursos que estão com o campo câmpus listado como "Câmpus Virtual de Ead" mas em seu nome estão vinculados a câmpus presenciais e o seu formato consta como "Presencial". 

Temporariamente eles serão excluídos do dataset resultante do pré-processamento até ser tomada uma decisão em relação ao formato de inclusão desses cursos, atividade que será realizada junto ao orientador.

In [69]:
cursos.drop(cursos[cursos["campus"] == "Câmpus Virtual de Ead"].index, inplace=True)

cursos.tail()

,campus,nome,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
252,VAL,Especialização em Ensino na Educação Básica,1427,Especialização,Presencial,Vespertino,Semestral,30,Pós-Graduação Lato Sensu,5316,420.0,NaN,NaN,NaN,NaN,420.0
253,VAL,Técnico Integrado ao Ensino Médio em Automação...,965,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3681,3240.0,120.0,200.0,54.0,NaN,3614.0
254,VAL,Técnico Integrado ao Ensino Médio em Mecânica,966,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3682,3240.0,120.0,200.0,NaN,NaN,3560.0
255,VAL,Técnico Integrado ao Ensino Médio em Eletrotéc...,932,Integrado - EJA,Presencial,Noturno,Semestral,60,Ensino Médio,3646,2160.0,240.0,200.0,NaN,NaN,2600.0
256,VAL,Licenciatura em Matemática,1014,Licenciatura,Presencial,Noturno,Anual,30,Superior,4422,2970.0,200.0,NaN,54.0,NaN,3224.0


Existem alguns cursos com o formato listado como "À Distância", uma solução para os cursos com o câmpus "Câmpus Virtual de Ead" seria pegar o campo câmpus deles e atualizar com a sigla do câmpus que é mencionado em seu nome e trocar o seu campo formato para "À distância".

In [ ]:
cursos[cursos["formato"] == "À Distância"].loc[:, ["campus", "nome", "codigo", "formato"]]

# Exportação dos dados pré-processados

In [ ]:
df_final = cursos

In [ ]:
df_final.head()

In [ ]:
export_path = Path(DA_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    df_final.to_csv(export_path / "cursos.csv", sep=";", index=False)
except Exception as e:
    print(str(e))